# Libraries and constants


In [0]:
#Required libraries

import math
import numpy as np
from sympy import *
from sympy import symbols
from sympy import diff
from scipy.integrate import quad
import matplotlib.pyplot as plt



#Constants

G = 6.67*10**(-11)
c = 3*10**8

m1 = 10**30/(c**2/G)
m2 = 10**30/(c**2/G)
mtot = m1 + m2
SNR = 10
f_upper = (((6**(3/2)) * np.pi * mtot * (G/(c*3)))**(-1))

s0 = 10**(-49)
fs = 20
f0 = 115

# User interface

This box is the interactive part of the code, as the user can tune it to achive desired outputs. The user can:

-assign different values to the observables (in the list "values_observables").

-select which parameters needs to be calculated in the standard deviation, just by setting them equal to "True" or "False"

-select which waveform take into account, just by changing the number (0,1,2) in the "waveform_choice" variable

-select which telescope/PSD take into account, just by changin the number (1,2) in the "psd_choice" variable

-create a new function in a simple way: it can be coded in "user_waveform(f, observables)"; if the user wants to have different parameters than the 4 standard observables, the user can just add/modify them to the boolean set, and modify their names in the list "all_observables_strings".

In [0]:
#Choose the observables for the Fisher Matrix 

values_observables = [1, 1, 1000, 1]         #change here the values of all the observables. They have the same order as the boolean variables right below.

std_tc = True          #Type True if you want to include this observable in the Fisher matrix, type False if you don't
std_phic = True
std_mchirp = True
std_eta = True

all_observables = [std_tc, std_phic, std_mchirp, std_eta]
all_observables_strings = ["Coalescence time", "Phase at coalescence instant", "Chirp mass", "Dimensionless mass ratio"]



#choose the PSD

psd_choice = 1
#Value == 1 allows to work with the ADV Ligo PSD.
#Value == 2 allows to work with the Einstein Telenscope PSD



#Choose the waveform

waveform_choice = 1
#The user can change this value to get different waveforms:
#Value == 1 allows to work with the first waveform, which is used for exercise 1;
#Value == 2 allows to work with the second waveform, which is for exercise 3;
#Value == 0 allows to work with a user-input waveform, which can be written just below.



#User-input waveform: the user can implement the function here

def user_waveform(f, observables):
  parameter1, parameter2, parameter3, parameter4 = values_observables

  pass

# Fisher Matrix Algorithm

In [0]:
#creation of the array with the observables selected for the Fisher matrix

all_observables_booleans = [int(std_tc), int(std_phic), int(std_mchirp), int(std_eta)]
selected_observables = [all_observables_strings[i]* all_observables_booleans[i] for i in range(0,len(all_observables_booleans))]
while '' in selected_observables:
    selected_observables.remove('')



#first waveform

alpha0 = 1

alpha1 = 0

alpha3 = (-16) * (np.pi)

def alpha2_function(eta):
    return (20/9) * (743/336) + (11/4)*eta

def alpha4_function(eta):
    return 10 * (3058673/1016064 + 5429/1008 * (eta) + 617/144 * (eta ** 2))

alpha2 = alpha2_function(eta)
alpha4 = alpha4_function(eta)


def v_function(f, observables):
    tc, phic, mchirp, eta = observables
    mtot = mchirp/(eta**(3/5))
    output = ( np.pi * mtot * f ) ** (1/3)
    return output

def sum_alphas_function(f, observables):
    tc, phic, mchirp, eta = observables
    v = v_function (f, observables)
    alphas = [alpha0, alpha1, alpha2, alpha3, alpha4]
    output = 0
    for i in range(len(alphas)):
        output += alphas[i] * (v ** (i))
    return output

def psi_function(f, observables):
    tc, phic, mchirp, eta = observables
    sum_alphas = sum_alphas_function(f, observables)
    v = v_function (f, observables)
    output = (2*np.pi*f*tc) - phic - (np.pi/4) + ((3/(128*eta*(v**5)))*(sum_alphas))
    return output

def psd1_function(f):
    if f >= fs:
      output = s0*(((f/f0)**(-4.14))-(5*(f/f0)**(-2))+((111*(1-(f/f0)**2+((f/f0)**4)/2))/(1+((f/f0)**2)/2)))
    else:
      output = float('inf')
    return output

def amplitude_function(f):
    s = psd1_function(f)              
    output = (np.abs((SNR)/(4*(quad((lambda f:((f*(-7/6))/s)), fs, f_upper)[0]))))**(1/2)       
    return output

def h1_function(f, observables):
  tc, phic, mchirp, eta = observables
  a = amplitude_function (f)
  psi = psi_function (f, observables)
  output = a * (f ** (-7/6)) * (np.exp(1j * psi))
  return output


#second waveform

def psi2_function(f, observables):
    tc, phic, mchirp, eta = observables
    sum_alphas = sum_alphas_function(f, observables)
    v = v_function (f, observables)
    output = (2*np.pi*f*tc) + phic - (np.pi/4) + ((3/(128*eta*(v**5)))*(sum_alphas))
    return output

def psd2_function(f):
  output = (8*np.exp(-40*(f**6))) - (2*np.exp(-35*f**4)) - (3*np.exp(-31*f**4)) - np.exp(-27*f*3) + (4 * np.exp(-24*f**2)) - (4 * np.exp(-21*f)) + (9* np.exp(-19))
  return output

def amplitude2_function(f):
    s = psd2_function(f)              
    output = (np.abs((SNR)/(4*(quad((lambda f:((f*(-7/6))/s)), fs, f_upper)[0]))))**(1/2)       
    return output

def h2_function(f, observables):
  tc, phic, mchirp, eta = observables
  a = amplitude2_function (f)
  psi = psi2_function (f, observables)
  output = a * (f ** (-7/6)) * (np.exp(1j * psi))
  return output



# the following function redirects the algorithm to the waveform and the PSD desired by the user

def h_function (f, observables):
    if waveform_choice == 1:
      output = h1_function (f, observables)
    if waveform_choice == 2:
      output = h2_function(f, observables)
    if waveform_choice == 0:
      output = user_waveform(f, observables)
    return output

def psd_function(f):
  if psd_choice == 1:
    output = psd1_function(f)
  if psd_choice == 2:
    output = psd2_function(f)
  return output


In [527]:
gamma_matrix = np.zeros([len(selected_observables),len(selected_observables)])

def derivative_function(f, observables, param_index):
  if param_index < len(values_observables):
    diff_values_observables=values_observables.copy()
    diff_values_observables[param_index] = (diff_values_observables[param_index] * 0.999)
    output = (h_function(f, values_observables) - h_function(f, diff_values_observables)) / (diff_values_observables[param_index]-(values_observables[param_index] * 0.9999))
  else:
    output = 1
    pass 
  return output


def gamma_element_function(f, observables, index1, index2):
  numerator = derivative_function(f, observables, index1) * np.conj(derivative_function(f, observables, index2))  #qua sto chiamando funzione quindi occhio
  denominator = psd_function(f)
  output = np.real(4*(quad((lambda f:(numerator/denominator)), fs, f_upper)[0]))
  return output


def gamma_matrix_function(f, observables):  #tutte variabili che passo a element? ma se parameters_choosen è variabile globale no     #values_observables                                                                
  row_num = 0
  column_num = 0
  i = 0
  while i < len(all_observables):          #all_observables (lista di tutte)
        if all_observables[i] == True:
            j = 0
            while j < len(all_observables):    #all_observables (lista di tutte)
                if all_observables[j] == True:     
                    gamma_matrix[row_num][column_num] = gamma_element_function(f, observables, i, j)
                    column_num = column_num + 1
                else:
                    pass
                j = j + 1
            column_num = 0
            row_num = row_num + 1
        else:
            pass
        i = i + 1
  output = gamma_matrix
  return output

fisher_matrix = gamma_matrix_function(f, values_observables)

def standard_deviations_function(f, observables):                 
  gamma_matrix_calculated = gamma_matrix_function(f, observables)
  inverted_matrix =  np.linalg.inv(gamma_matrix_calculated)
  i = 0
  output = []
  while i < len(selected_observables):                                  
    output.append((np.abs(inverted_matrix.item(i,i)))**(1/2))
    i = i + 1
  return output

                                                                                            
final_results_list = standard_deviations_function(f, values_observables)          
print("For", selected_observables, "variances are: ", final_results_list, "respectively. \n\n\n\n\n\n\n")                                 

For ['Coalescence time', 'Phase at coalescence instant', 'Chirp mass', 'Dimensionless mass ratio'] variances are:  [165683548240.15594, 5.298101605667723e+16, 5.217527130133113e+19, 1.6619366731980926e+16] respectively. 









/usr/local/lib/python3.6/dist-packages/scipy/integrate/quadpack.py:453: ComplexWarning: Casting complex values to real discards the imaginary part
  return _quadpack._qagse(func,a,b,args,full_output,epsabs,epsrel,limit)


In [0]:
ADV_LIGO = [165683548240.15594, 5.298101605667723e+16, 5.217527130133113e+19, 1.6619366731980926e+16]

ET = [155518297652.74948, 2.5722245934939624e+16, 3.6893488147419103e+19, 1.320358031285309e+16]

Parameters = ["Coalescence time", "Phase at coalescence instant", "Chirp mass", "Dimensionless mass ratio"]

PSD function of the Einstein Telescope:

![alt text](https://drive.google.com/uc?id=1fgSOkTkWor7xFswSmnKTddDie3_TXtQf)



Relation between the accuracy of the ADV_LIGO Telescope (1) and the Einstein Telescope (2).

![alt text](https://drive.google.com/uc?id=1BXMqfoN7A_AAMqzcEjN7NAxvyvQYmWX0)

(values in the graphs are taked from:
ADV_LIGO = [165683548240.15594, 5.298101605667723e+16, 5.217527130133113e+19, 1.6619366731980926e+16]

ET = [155518297652.74948, 2.5722245934939624e+16, 3.6893488147419103e+19, 1.320358031285309e+16])




